In [2]:
import numpy as np
from random import shuffle
 
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [5]:
len(train_input)

1048576

In [8]:
train_input[0].shape

(20, 1)

In [9]:
train_output = []
 
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)


In [12]:
len(train_output)

1048576

In [15]:
len(train_output[0])

21

In [16]:
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

In [17]:
import tensorflow as tf

In [18]:
data = tf.placeholder(tf.float32, [None, 20,1])
target = tf.placeholder(tf.float32, [None, 21])

In [19]:
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

In [21]:
len(train_input)

10000

In [23]:
data = tf.placeholder(tf.float32, [None, 20,1]) #Number of examples, number of input, dimension of each input
target = tf.placeholder(tf.float32, [None, 21])

In [24]:
data

<tf.Tensor 'Placeholder_4:0' shape=(?, 20, 1) dtype=float32>

In [25]:
target

<tf.Tensor 'Placeholder_5:0' shape=(?, 21) dtype=float32>

In [26]:
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [27]:
val

<tf.Tensor 'rnn/transpose_1:0' shape=(?, 20, 24) dtype=float32>

In [28]:
_

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 24) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 24) dtype=float32>)

In [29]:
val = tf.transpose(val, [1, 0, 2])

In [30]:
val

<tf.Tensor 'transpose:0' shape=(20, ?, 24) dtype=float32>

In [33]:
val.get_shape()[0]

Dimension(20)

In [31]:
last = tf.gather(val, int(val.get_shape()[0]) - 1)

In [32]:
last

<tf.Tensor 'Gather:0' shape=(?, 24) dtype=float32>

In [34]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

In [35]:
target.get_shape()[1]

Dimension(21)

In [36]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [37]:
prediction

<tf.Tensor 'Softmax:0' shape=(?, 21) dtype=float32>

In [38]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

c:\users\gebruiker\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [41]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

batch_size = 1000
no_of_batches = int(len(train_input)) / batch_size
epoch = 5000
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print("Epoch ",str(i))
incorrect = sess.run(error,{data: test_input, target: test_output})
print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


TypeError: 'float' object cannot be interpreted as an integer